In [1]:
import pandas as pd
import numpy as np
import wget
import os

In [2]:
county_data = pd.read_csv("us-counties.csv")

FileNotFoundError: [Errno 2] File b'us-counties.csv' does not exist: b'us-counties.csv'

In [61]:
nulls = county_data[county_data.isna().any(axis=1)]

In [73]:
nulls.to_csv("nulls.csv")
# All rows where FIPS is null. This consists of (a) New York City (all reported as one unit), (b) various Unknowns. 
# See NYT documentation on geographical exceptions.

In [94]:
# Download daily data files -- did this on 5/20. Saved as .txt files in /data

for m in range(4):
    for d in range(31):
        month = str(m+2).zfill(2)
        day = str(d+1).zfill(2)
        date = "2020"+month+day
        url = "https://s3-us-west-1.amazonaws.com//files.airnowtech.org/airnow/2020/"+date+"/daily_data_v2.dat"
        try:
            wget.download(url, "./data/"+date+".txt")
            print('Downloading', date)
        except:
            print('No file for date', date)

No file for date 20200230
No file for date 20200231
No file for date 20200431
No file for date 20200521
No file for date 20200522
No file for date 20200523
No file for date 20200524
No file for date 20200525
No file for date 20200526
No file for date 20200527
No file for date 20200528
No file for date 20200529
No file for date 20200530
No file for date 20200531


In [20]:
# make DFs from .txt files
# date_dfs is a dictionary of DFs where keys are dates in MMDD format

# DF column values: "aqsid" = site ID, first 5 digits are FIPS code; 
# "period" = averaging period (either peak 1-hr, peak 8-hr average, or 24-hr average);
# "aqi_value" = 8-hr or 24-hr AQI value, -999 = peak 1-hr ozone or peak 24-hr SO2; "aqi_category" = 0=good etc.;
# "aqsid_full" = AQS site ID preceded by 3-digit country code

date_dfs = {}
allfiles = os.listdir('data')[1:]
for filename in allfiles:
    date = filename[4:8]
    df = pd.read_csv('data/'+filename, sep="|", header=None)
    df.columns = ["date","aqsid","sitename","parameter","units","value","period","source","aqi_value","aqi_category","latitude","longitude","aqsid_full"]
    df['fips'] = df['aqsid'].str.slice(stop=5)  # add FIPS column based on first 5 digits of AQSID
    del df['aqsid']
    del df['source']
    del df['latitude']
    del df['longitude']
    df = df.loc[df['aqsid_full'].str[:3] == '840']   # delete non-US rows 
    del df['aqsid_full']
    df.to_csv('air_quality_tables/'+filename[:-3]+'csv')
    date_dfs[date] = df

In [170]:
date_dfs['0520'] = None   # dropped today's table because it contained data from 5/18 for some reason

In [28]:
master_df = pd.concat(date_dfs.values())

In [40]:
dates = master_df['date'].str
newdates = "20"+dates[6:8]+"-"+dates[3:5]+"-"+dates[0:2]

In [43]:
master_df['date'] = newdates

In [45]:
master_df.to_csv('all_data.csv')

In [46]:
master_df.groupby(['parameter']).count()

,date,sitename,units,value,period,aqi_value,aqi_category,fips
parameter,,,,,,,,
CO-8hr,17375,17375,17375,17375,17375,17375,17375,17375
OZONE-1HR,111450,111450,111450,111450,111450,111450,111450,111450
OZONE-8HR,106931,106931,106931,106931,106931,106931,106931,106931
PM10-24hr,29709,29709,29709,29709,29709,29709,29709,29709
PM2.5-24hr,89894,89892,89894,89894,89894,89894,89894,89894
SO2-24HR,25557,25557,25557,25557,25557,25557,25557,25557


In [52]:
names = master_df['parameter'].unique().tolist()
ozone1 = master_df[master_df['parameter']==names[0]]
ozone8 = master_df[master_df['parameter']==names[1]]
pm10 = master_df[master_df['parameter']==names[2]]
pm25 = master_df[master_df['parameter']==names[3]]
co8 = master_df[master_df['parameter']==names[4]]
so2 = master_df[master_df['parameter']==names[5]]

# deleted unneeded columns and saved each as CSV

In [6]:
# table of FIPS codes to county/state

fips = pd.read_html('https://www.nrcs.usda.gov/wps/portal/nrcs/detail/national/home/?cid=nrcs143_013697')[0]

In [7]:
fips = fips.drop(3232) # last row was gibberish
# did some other massaging which I deleted the code for

In [48]:
fips.to_csv('fips.csv')

In [72]:
fips = pd.read_csv('fips.csv', dtype={'fips':"str"})
del fips['Unnamed: 0']

# for some reason the CSV loses leading zeroes in the fips column; 
# opening like this fixes it for the dataframe